## Demo: ShiftAmountActivity
The basic steps to set up an OpenCLSim simulation are:
* Import libraries
* Initialise simpy environment
* Define object classes
* Create objects
  * Create sites
  * Create vessels
  * Create activities
* Register processes and run simpy

----

This notebook shows the workings of the ShiftAmountActivity. This activity uses a processor to transfer a specified number of objects from an origin resource, which must have a container, to a destination resource, which also must have a container. In this case it shifts payload from a from_site to vessel01.

NB: The ShiftAmountActivity checks the possible amount of objects which can be transferred, based on the number of objects available in the origin, the number of objects which can be stored in the destination and the number of objects requested to be transferred. If the number of actually to be transferred objects is zero than an exception is raised. These cases have to be prevented by using appropriate events.

#### 0. Import libraries

In [1]:
import datetime, time
import simpy

import shapely.geometry
import pandas as pd

import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

#### 1. Initialise simpy environment

In [2]:
# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)

#### 2. Define object classes

In [3]:
# create a Site object based on desired mixin classes
Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasContainer,
        core.HasResource,
    ),
    {},
)

# create a TransportProcessingResource object based on desired mixin classes
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,
        core.Log,
        core.ContainerDependentMovable,
        core.HasResource,
        core.Processor,
    ),
    {},
)

#### 3. Create objects
##### 3.1. Create site object(s)

In [4]:
# prepare input data for from_site
location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)
data_from_site = {"env": my_env,
                "name": "from_site",
                "geometry": location_from_site,
                "capacity": 100,
                "level": 100
               }
# instantiate to_site 
from_site = Site(**data_from_site)

##### 3.2. Create vessel object(s)

In [5]:
# prepare input data for vessel_01
data_vessel01 = {"env": my_env,
                 "name": "vessel01",
                 "geometry": location_from_site, 
                 "capacity": 5,
                 "compute_v": lambda x: 10
               }
# instantiate vessel_01 
vessel01 = TransportProcessingResource(**data_vessel01)

##### 3.3 Create activity/activities

In [6]:
# initialise registry
registry = {}

In [7]:
shift_amount_activity_data = model.ShiftAmountActivity(
    env=my_env,
    name="Shift amount activity",
    registry=registry,
    processor=vessel01,
    origin=from_site,
    destination=vessel01,
    amount=100,
    duration=60,
)

#### 4. Register processes and run simpy

In [8]:
model.register_processes([shift_amount_activity_data])
my_env.run()

#### 5. Inspect results
##### 5.1 Inspect logs
We can now inspect the logs. The model now shifted cargo from the from_site onto vessel01.

In [9]:
display(plot.get_log_dataframe(shift_amount_activity_data, [shift_amount_activity_data]))

,Activity,Timestamp,ActivityState
0,Shift amount activity,1970-01-01 00:00:00,START
1,Shift amount activity,1970-01-01 00:01:00,STOP


In [10]:
display(plot.get_log_dataframe(from_site, [shift_amount_activity_data]))

,Activity,Timestamp,ActivityState,container level,geometry
0,Shift amount activity,1970-01-01 00:00:00,START,100.0,POINT (4.18055556 52.18664444)
1,Shift amount activity,1970-01-01 00:01:00,STOP,95.0,POINT (4.18055556 52.18664444)


In [11]:
display(plot.get_log_dataframe(vessel01, [shift_amount_activity_data]))

,Activity,Timestamp,ActivityState,geometry,container level
0,Shift amount activity,1970-01-01 00:00:00,START,POINT (4.18055556 52.18664444),0.0
1,Shift amount activity,1970-01-01 00:01:00,STOP,POINT (4.18055556 52.18664444),5.0


Observe that an amount has been shifted from from_site to vessel01. There was no movement.